In [1]:
# Import the necessary libraries
import pyazr
import emcee

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm
from brick.azr import AZR as brick

In [2]:
# Define the data labels (in AZURE2 order)
labels = ["Meyer et al. (1976) - 84.3 deg",
          "Meyer et al. (1976) - 114.5 deg",
          "Meyer et al. (1976) - 144.1 deg", 
          "LUNA HPGe (2023)", 
          "LUNA BGO (2023)",
          "Felsenkeller (2023)",
          "ATOMKI (2023)",
          "Notre Dame (2023) - 0 deg",
          "Notre Dame (2023) - 55 deg",
          "Burtebaev et al. (2008)",
          "Lamb et al. (1957)",
          "Bailey et al. (1950)",
          "Vogl et al. (1963)",
          "Rolfs et al. (1974) - 0 deg",
          "Rolfs et al. (1974) - 90 deg"]

In [6]:
#backend = emcee.backends.HDFBackend("results/mcmc/samples_norms.h5")

# Get the number of samples after burn in
#nb  = 200
#lnp = backend.get_log_prob(discard=nb)
#ii  = np.where(np.median(lnp, axis=0)!=-np.inf)[0]

# Get only the first chain
#data_mcmc = backend.get_chain(discard=nb, thin=20)[:, ii, :]
#data_mcmc = data_mcmc.reshape((-1, data_mcmc.shape[-1]))

# Shuffle the data to get a better representation
#np.random.shuffle( data_mcmc )
#data_mcmc = data_mcmc[:1000]
import pickle as pkl

with open("results/bilby/ntemps_5_checkpoint_resume.pickle", "rb") as f:
    dump = pkl.load(f)

samples = dump["chain_array"][:,500:].flatten()

# Create a numpy array of the data
data = np.zeros( (int(len(samples)/28), 28) )
for i in range( int(len(samples)/28) ):
    if( samples[28*i] == 0 ): continue
    data[i] = samples[28*i:28*(i + 1)]

# Remove all rows with 0
data = data[~np.all(data == 0, axis=1)]

np.random.shuffle( data )
data_mcmc = data[:1000]

In [7]:
data_freq = []
files = glob( "results/norms/samples_*.txt" )
for file in files:
    data_freq.append( np.loadtxt( file ) )
data_freq = np.concatenate( data_freq, axis=0 )[:1000]

In [5]:
# Prepare pyazr to extrapolate the data
azr = pyazr.azure2( "12c_pg.azr" )
azr.extrap_mode( )
segments = azr.nsegments

# Get the lables
brk = brick( "12c_pg.azr" )
params_labels = brk.config.labels
params_labels[-len(labels):] = labels

In [8]:
# Plot the histograms of data_mcmc and data_freq on the same plot for each parameter
for i in range( data_mcmc.shape[1] ):
    fig, ax = plt.subplots( 1, 1, figsize=( 6, 4 ) )
    sns.histplot( data_mcmc[:, i], kde=True, ax=ax, color="blue", label="MCMC" )
    sns.histplot( data_freq[:, i], kde=True, ax=ax, color="red", label="Frequentist" )
    #ax.set_xlabel( params_labels[i] )
    ax.legend( )
    #ax.set_yscale( "log" )
    plt.savefig( "results/comparison/parameter_{}.pdf".format( i ) )
    plt.close( )